In [ ]:
# Initial imports
import warnings
warnings.filterwarnings('ignore')
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
# Initialize the Panel Extensions (for Plotly)
import panel as pn
import param
pn.extension('plotly')
pn.extension()
import plotly.express as px
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline
#!pip install yfinance
import yfinance as yf
#!pip install pandas-montecarlo
import pandas_montecarlo
import datetime as dt
from MCForecastTools import MCSimulation

# Load .env enviroment variables
load_dotenv()



In [51]:
# Define Parameters

range_of_cryptos = ["BTC-USD", "ETH-USD", "BNB-USD", "ADA-USD", "DOGE-USD", "XRP-USD", "LINK-USD" , "THETA-USD", "LTC-USD", "XLM-USD"]
range_of_stocks = ["AMZN", "TSLA", "NEO", "AAPL", "NVDA", "BABA", "NFLX", "DIS", "NKE"]
#num_sim = 10
# Define Functions

#CALCULATE 1 YEARS BACK DATE
def calc_1years_backdate():
    end = dt.datetime.now().date()
    start = end - dt.timedelta(days=365)
    return start
#CALCULATE 2 YEARS BACK DATE
def calc_2years_backdate():
    end = dt.datetime.now().date()
    start = end - dt.timedelta(days=730)
    return start
#CALCULATE 3 YEARS BACK DATE
def calc_3years_backdate():
    end = dt.datetime.now().date()
    start = end - dt.timedelta(days=1095)
    return start


#download crypto data and stock data for 3 years
def download_cryptos_data():
    df = yf.download(range_of_cryptos, period = "3y", group_by = 'ticker', threads = True, progress=False)
    df = df.dropna().copy()
    return df

def download_stocks_data():
    df = yf.download(range_of_stocks, period = "3y", group_by = 'ticker', threads = True, progress=False)
    df = df.dropna().copy()
    return df

##annual return function:
def daily_return_data(df):
    daily_return = df["Close"].pct_change()
    plot = daily_return.hvplot.line(x="Date", rot=45, shared_axes=False).opts(yformatter="%.0f", width=500, xlabel="Time", ylabel="Price", title="Daily Return")
    return plot



def cumulative_return_plot(df):
    cumulative_returns = (1 + df['Close'].pct_change()).cumprod()
    plot = cumulative_returns.hvplot.line(x="Date", rot=45, shared_axes=False).opts(yformatter="%.0f", width=500, xlabel="Time", ylabel="Price", title="Cumulative Return")
    return plot


def MA_plot(df):
    df["30MA"]=df[["Close"]].rolling(30).mean()
    df["60MA"]=df[["Close"]].rolling(60).mean()
    df["90MA"]=df[["Close"]].rolling(90).mean()
    df.dropna(inplace=True)
    plot = df[["Close", "30MA", "60MA", "90MA"]].hvplot.line(x="Date", rot=45, shared_axes=False).opts(yformatter="%.0f", width=500, xlabel="Date", ylabel="Price", title="Moving Average")   
    return plot

#Calculate Sharpe Ratios
def daily_returns_multiple_tickers(df):
    df2 = pd.DataFrame(columns = df.columns.levels[0])
    df = df.pct_change()
    df = df.dropna()
    for key in (df.columns.levels[0]):
        df2[key] =+ (df[key]["Close"])
    return df2  
def annual_sharpe_ratio_plot(df):
    df = daily_returns_multiple_tickers(df)
    std = df.std()
    annual_std = std * np.sqrt(252)
    annual_sharpe_ratio = (df.mean() * 252) / annual_std
    plot = annual_sharpe_ratio.hvplot.bar(rot=45, shared_axes=False).opts(width=500, xlabel="Date", ylabel="Price", title="Annual Sharpe Ratios")
    return plot


# Calculate Monte Carlo cumulative return of a dataframe
def mc_cumulative_return(df, num_sim, num_day):
    mc = MCSimulation(
        portfolio_data = df,
        weights = "",
        num_simulation = num_sim,
        num_trading_days = num_day
    )
    return mc.calc_cumulative_return()

def mc_plot_cumulative(df, num_sim, num_day):
    mc = MCSimulation(
        portfolio_data = df,
        weights = "",
        num_simulation = num_sim,
        num_trading_days = num_day
    )
    plot = mc.plot_simulation()
    return plot

def mc_plot_distribution(df, num_sim, num_day):
    mc = MCSimulation(
        portfolio_data = df,
        weights = "",
        num_simulation = num_sim,
        num_trading_days = num_day
    )
    plot = mc.plot_distribution()
    return plot

def mc_summarize_statistic(df, num_sim, num_day, initial_investment):
    mc = MCSimulation(
        portfolio_data = df,
        weights = "",
        num_simulation = num_sim,
        num_trading_days = num_day
    )
    tbl = mc.summarize_cumulative_return()
    ci_lower = round(tbl[8]*initial_investment,2)
    ci_upper = round(tbl[9]*initial_investment,2)
    return (f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio over the next {num_day} days will end within in the range of ${ci_lower} and ${ci_upper}")
# Example: mc_cumulative_return(crypto_df[["BTC-USD"]], 10, 365)
# Example: mc_plot_distribution(crypto_df[["BTC-USD"]], 100, 365)
# Example: mc_summarize_statistic(crypto_df[["BTC-USD"]], 100, 365, 20000)


In [52]:
# Create a Title for the Dashboard

# Fetch initial data:
crypto_df = download_cryptos_data()
stock_df = download_stocks_data()
fetch_data_button = pn.widgets.Button(name='FETCH HISTORICAL DATA FROM YAHOO FINANCE!', button_type='success')
def update_data(event):
    crypto_df = download_cryptos_data()
    stock_df = download_stocks_data()
fetch_data_button.on_click(update_data)



# Define title and welcome text
title = "#Investment Projection Dasboard"
welcome = "This Dashboard provice a visual analysis of historial value for certain stock and cryptos"


#Create Widget Elements:

crypto_ticker = pn.widgets.Select(name='Crypto Tickes', options=range_of_cryptos)
crypto_year_slider = pn.widgets.IntSlider(name='Years of Crypto Data', start=1, end=3, step=1, value=1)
stock_ticker = pn.widgets.Select(name='Stock Tickers', options=range_of_stocks) 
stock_year_slider = pn.widgets.IntSlider(name='Years of Stock Data', start=1, end=3, step=1, value=1)

# Input for Historical data
#Crypto historical Data:
crypto_input = pn.Column(crypto_ticker, crypto_year_slider, align='center')
@pn.depends(crypto_ticker, crypto_year_slider)
def reactive_crypto_MAchart(crypto_ticker, crypto_year_slider):
    if crypto_year_slider == 1:
        return MA_plot(crypto_df[crypto_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif crypto_year_slider == 2:
        return MA_plot(crypto_df[crypto_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return MA_plot(crypto_df[crypto_ticker])
    
@pn.depends(crypto_ticker, crypto_year_slider)
def reactive_crypto_cumulative(crypto_ticker, crypto_year_slider):
    if crypto_year_slider == 1:
        return cumulative_return_plot(crypto_df[crypto_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif crypto_year_slider == 2:
        return cumulative_return_plot(crypto_df[crypto_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return cumulative_return_plot(crypto_df[crypto_ticker])
crypto_column = pn.Column(crypto_input, reactive_crypto_MAchart, reactive_crypto_cumulative)


#Stock historical Data:
stock_input = pn.Column(stock_ticker, stock_year_slider, align='center')
@pn.depends(stock_ticker, stock_year_slider)
def reactive_stock_MAchart(stock_ticker, stock_year_slider):
    if stock_year_slider == 1:
        return MA_plot(stock_df[stock_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif stock_year_slider == 2:
        return MA_plot(stock_df[stock_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return MA_plot(stock_df[stock_ticker])

@pn.depends(stock_ticker, stock_year_slider)
def reactive_stock_cumulative(stock_ticker, stock_year_slider):
    if stock_year_slider == 1:
        return cumulative_return_plot(stock_df[stock_ticker].loc[calc_1years_backdate():dt.datetime.now().date()])
    elif stock_year_slider == 2:
        return cumulative_return_plot(stock_df[stock_ticker].loc[calc_2years_backdate():dt.datetime.now().date()])
    else:
        return cumulative_return_plot(stock_df[stock_ticker])
stock_column = pn.Column(stock_input, reactive_stock_MAchart, reactive_stock_cumulative)

historical_data_row = pn.Row(crypto_column, stock_column, width=1100)


#Input for your proforlio
crypto_investment_ticker = pn.widgets.Select(name='Crypto Tickes', options=range_of_cryptos)
crypto_investment_amt = pn.widgets.FloatInput(name='Your Crypto Investment in USD', value=0)
crypto_future_slider = pn.widgets.IntSlider(name='Days of Monte Carlo projection', start=0, end=730, step=365, value=0)
crypto_sim_num = pn.widgets.IntInput(name='Number of Simulation', value=10, step=10, start=10, end=1000)
#cal_crypto_MC_button = pn.widgets.Button(name='Update Simulation Crypto MC', button_type='danger')


stock_investment_ticker = pn.widgets.Select(name='Stock Tickes', options=range_of_stocks)
stock_investment_amt = pn.widgets.FloatInput(name='Your Stock Investment in USD', value=0)
stock_future_slider = pn.widgets.IntSlider(name='Days of Monte Carlo projection', start=0, end=730, step=365, value=0)
stock_sim_num = pn.widgets.IntInput(name='Number of Simulation', value=10, step=10, start=10, end=1000)
#cal_stock_MC_button = pn.widgets.Button(name='Update Simulation Crypto MC', button_type='danger')

    

@pn.depends(crypto_investment_ticker, crypto_sim_num, crypto_future_slider)
def disp_crypto_MC_return_data(crypto_investment_ticker, crypto_sim_num, crypto_future_slider):
    if crypto_future_slider > 0:
        return mc_plot_cumulative(crypto_df[[crypto_investment_ticker]], crypto_sim_num, crypto_future_slider)
    else:
        return mc_plot_cumulative(crypto_df[[crypto_investment_ticker]], crypto_sim_num, 0)


@pn.depends(crypto_investment_ticker, crypto_sim_num, crypto_future_slider)
def disp_crypto_MC_distribution_data(crypto_investment_ticker, crypto_sim_num, crypto_future_slider):
    if crypto_future_slider > 0:
        return mc_plot_distribution(crypto_df[[crypto_investment_ticker]], crypto_sim_num, crypto_future_slider)
    else:
        return mc_plot_distribution(crypto_df[[crypto_investment_ticker]], crypto_sim_num, 0)
#-----
@pn.depends(crypto_investment_ticker, crypto_sim_num, crypto_future_slider, crypto_investment_amt)
def disp_crypto_MC_summarize(crypto_investment_ticker, crypto_sim_num, crypto_future_slider, crypto_investment_amt):
    if crypto_future_slider > 0:
        return mc_summarize_statistic(crypto_df[[crypto_investment_ticker]], crypto_sim_num, crypto_future_slider, crypto_investment_amt)
    else:
        return mc_summarize_statistic(crypto_df[[crypto_investment_ticker]], crypto_sim_num, 0, 0)
    
@pn.depends(stock_investment_ticker, stock_sim_num, stock_future_slider)
def disp_stock_MC_return_data(stock_investment_ticker, stock_sim_num, stock_future_slider):
    if stock_future_slider > 0:
        return mc_plot_cumulative(stock_df[[stock_investment_ticker]], stock_sim_num, stock_future_slider)
        
    else:
        return mc_plot_cumulative(stock_df[[stock_investment_ticker]], stock_sim_num, 0)
    
@pn.depends(stock_investment_ticker, stock_sim_num, stock_future_slider)
def disp_stock_MC_distribution_data(stock_investment_ticker, stock_sim_num, stock_future_slider):
    if stock_future_slider > 0:
        return mc_plot_distribution(stock_df[[stock_investment_ticker]], stock_sim_num, stock_future_slider)
        
    else:
        return mc_plot_distribution(stock_df[[stock_investment_ticker]], stock_sim_num, 0)
    
@pn.depends(stock_investment_ticker, stock_sim_num, stock_future_slider, stock_investment_amt)
def disp_stock_MC_summarize(stock_investment_ticker, stock_sim_num, stock_future_slider, stock_investment_amt):
    if stock_future_slider > 0:
        return mc_summarize_statistic(stock_df[[stock_investment_ticker]], stock_sim_num, stock_future_slider, stock_investment_amt)
    else:
        return mc_summarize_statistic(stock_df[[stock_investment_ticker]], stock_sim_num, 0, 0)
    
    
crypto_user_input = pn.Column(crypto_investment_ticker, crypto_sim_num, crypto_future_slider, crypto_investment_amt, disp_crypto_MC_summarize, disp_crypto_MC_return_data, disp_crypto_MC_distribution_data, align='center')
stock_user_input = pn.Column(stock_investment_ticker, stock_sim_num, stock_future_slider, stock_investment_amt, disp_stock_MC_summarize, disp_stock_MC_return_data, disp_stock_MC_distribution_data, align='center')
user_input = pn.Row(crypto_user_input, stock_user_input)



# Create a layout for the dashboard
dashboard = pn.WidgetBox(pn.Column(title, fetch_data_button,
                        pn.WidgetBox(pn.Tabs(    
                            ("Historical Data", historical_data_row),
                            ("MC Projection Alnalysis", user_input),
                            )), align="center"))

dashboard.servable()


WidgetBox
    [0] Column(align='center')
        [0] Markdown(str)
        [1] Button(button_type='success', name='FETCH HISTORICAL D...)
        [2] WidgetBox
            [0] Tabs
                [0] Row(width=1100)
                    [0] Column
                        [0] Column(align='center')
                            [0] Select(name='Crypto Tickes', options=['BTC-USD', 'ETH-USD', ...], value='BTC-USD')
                            [1] IntSlider(end=3, name='Years of Crypto Data', start=1, value=1)
                        [1] ParamFunction(function)
                        [2] ParamFunction(function)
                    [1] Column
                        [0] Column(align='center')
                            [0] Select(name='Stock Tickers', options=['AMZN', 'TSLA', ...], value='AMZN')
                            [1] IntSlider(end=3, name='Years of Stock Data', start=1, value=1)
                        [1] ParamFunction(function)
                        [2] ParamFunction(function)
                [1] Row
                    [0] Column(align='center')
                        [0] Select(name='Crypto Tickes', options=['BTC-USD', 'ETH-USD', ...], value='BTC-USD')
                        [1] IntInput(end=1000, name='Number of Simulation', start=10, step=10, value=10)
                        [2] IntSlider(end=730, name='Days of Monte C..., step=365)
                        [3] FloatInput(name='Your Crypto I...)
                        [4] ParamFunction(function)
                        [5] ParamFunction(function)
                        [6] ParamFunction(function)
                    [1] Column(align='center')
                        [0] Select(name='Stock Tickes', options=['AMZN', 'TSLA', ...], value='AMZN')
                        [1] IntInput(end=1000, name='Number of Simulation', start=10, step=10, value=10)
                        [2] IntSlider(end=730, name='Days of Monte C..., step=365)
                        [3] FloatInput(name='Your Stock Investment i...)
                        [4] ParamFunction(function)
                        [5] ParamFunction(function)
                        [6] ParamFunction(function)

In [ ]:
button = pn.widgets.Button(name='Click me', button_type='primary')

text = pn.widgets.TextInput(value='Ready')

def b(a):
    text.value = 'Clicked {0} times'.format(button.clicks)
    
button.on_click(b)
pn.Row(button, text)

In [ ]:

crypto_investment_ticker = "BTC-USD"
crypto_future_slider = 365

def calc_crypto_MC_data(crypto_investment_ticker, crypto_future_slider):
    if cal_crypto_MC_button.value == True:
        return mc_plot_cumulative(crypto_df[[crypto_investment_ticker]], crypto_future_slider)
      

pn.Row(cal_crypto_MC_button, calc_crypto_MC_data)

#mc_plot_cumulative(crypto_df[[crypto_investment_ticker]], crypto_future_slider)

In [16]:
crypto_sim_num = pn.widgets.IntInput(name='Number of Simulation', value=10, step=10, start=10, end=1000)
print(crypto_sim_num.value)

10


In [ ]:
button = pn.widgets.Button(name='Click me', button_type='primary')
text = pn.widgets.TextInput(value='Ready')

def b(event):
    print(button.clicks)
    
button.on_click(b)
pn.Row(button, text)